In [1]:
import os
import json
import h5py

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
# solve the problem of "libdevice not found at ./libdevice.10.bc"
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/home/r10222035/.conda/envs/tf2'

2025-10-08 10:23:57.542056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 10:23:57.624997: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Apply $H \to \gamma\gamma$ classifier to $H \to ZZ \to 4\ell$ events

In [2]:
def create_test_sample_from(npy_dirs: list, nevents: tuple, ratios=(0.8, 0.2), seed=0):
    # npy_dirs: list of npy directories
    # nevents: tuple of (n_VBF_SR, n_VBF_BR, n_GGF_SR, n_GGF_BR)
    # ratios: tuple of (r_train, r_val)

    npy_dir0 = Path(npy_dirs[0])

    data_VBF_SR = np.load(npy_dir0 / 'VBF_in_SR-data.npy')
    data_VBF_BR = np.load(npy_dir0 / 'VBF_in_BR-data.npy')
    data_GGF_SR = np.load(npy_dir0 / 'GGF_in_SR-data.npy')
    data_GGF_BR = np.load(npy_dir0 / 'GGF_in_BR-data.npy')

    n_VBF_SR, n_GGF_SR, n_VBF_BR, n_GGF_BR = nevents
    n_test = 10000
    n_VBF_SR_test = int(data_VBF_SR.shape[0] / (data_VBF_SR.shape[0] + data_VBF_BR.shape[0]) * n_test)
    n_VBF_BR_test = n_test - n_VBF_SR_test
    n_GGF_SR_test = int(data_GGF_SR.shape[0] / (data_GGF_SR.shape[0] + data_GGF_BR.shape[0]) * n_test)
    n_GGF_BR_test = n_test - n_GGF_SR_test

    np.random.seed(seed)
    # print(data_GGF_SR.shape[0], data_GGF_BR.shape[0], data_VBF_SR.shape[0], data_VBF_BR.shape[0])
    idx_VBF_SR = np.random.choice(data_VBF_SR.shape[0], n_VBF_SR + n_VBF_SR_test, replace=False)
    idx_VBF_BR = np.random.choice(data_VBF_BR.shape[0], n_VBF_BR + n_VBF_BR_test, replace=False)
    idx_GGF_SR = np.random.choice(data_GGF_SR.shape[0], n_GGF_SR + n_GGF_SR_test, replace=False)
    idx_GGF_BR = np.random.choice(data_GGF_BR.shape[0], n_GGF_BR + n_GGF_BR_test, replace=False)

    idx_VBF_SR_te = idx_VBF_SR[n_VBF_SR:]
    idx_VBF_BR_te = idx_VBF_BR[n_VBF_BR:]
    idx_GGF_SR_te = idx_GGF_SR[n_GGF_SR:]
    idx_GGF_BR_te = idx_GGF_BR[n_GGF_BR:]

    print(f'Preparing dataset from {npy_dirs}')

    new_data_te = np.concatenate([
        data_VBF_SR[idx_VBF_SR_te],
        data_VBF_BR[idx_VBF_BR_te],
        data_GGF_SR[idx_GGF_SR_te],
        data_GGF_BR[idx_GGF_BR_te],
    ], axis=0)
    data_te = new_data_te

    new_label_te = np.zeros(new_data_te.shape[0])
    new_label_te[:n_test] = 1
    label_te = new_label_te

    return data_te, label_te

In [3]:
def compute_nevent_in_SR_BR(GGF_cutflow_file='../Sample/selection_results_GGF_300_3.1.npy', VBF_cutflow_file='../Sample/selection_results_VBF_300_3.1.npy', L=300, cut_type='mjj'):
    # https://twiki.cern.ch/twiki/bin/view/LHCPhysics/CERNYellowReportPageAt14TeV
    cross_section_GGF = 54.67 * 1000
    cross_section_VBF = 4.278 * 1000
    # https://twiki.cern.ch/twiki/bin/view/LHCPhysics/CERNYellowReportPageBR
    BR_Haa = 0.00227

    GGF_selection = np.load(GGF_cutflow_file, allow_pickle=True).item()
    VBF_selection = np.load(VBF_cutflow_file, allow_pickle=True).item()

    if cut_type == 'quark_jet_2':
        n_GGF_SR = cross_section_GGF * GGF_selection['cutflow_number']['two quark jet: sig region'] / GGF_selection['cutflow_number']['Total'] * BR_Haa * L
        n_GGF_BR = cross_section_GGF * GGF_selection['cutflow_number']['two quark jet: bkg region'] / GGF_selection['cutflow_number']['Total'] * BR_Haa * L
        n_VBF_SR = cross_section_VBF * VBF_selection['cutflow_number']['two quark jet: sig region'] / VBF_selection['cutflow_number']['Total'] * BR_Haa * L
        n_VBF_BR = cross_section_VBF * VBF_selection['cutflow_number']['two quark jet: bkg region'] / VBF_selection['cutflow_number']['Total'] * BR_Haa * L

    else:
        raise ValueError('cut_type must be mjj, deta, or mjj, or deta, or gluon_jet')
    return n_VBF_SR, n_GGF_SR, n_VBF_BR, n_GGF_BR

In [4]:
def get_sample_size(y):
    if len(y.shape) == 1:
        ns = (y == 1).sum()
        nb = (y == 0).sum()
    else:
        ns = (y.argmax(axis=1) == 1).sum()
        nb = (y.argmax(axis=1) == 0).sum()
    print(ns, nb)
    return ns, nb

In [5]:
def pt_normalization(X):
    # input shape: (n, res, res, 3)
    mean = np.mean(X, axis=(1, 2), keepdims=True)
    std = np.std(X, axis=(1, 2), keepdims=True)
    epsilon = 1e-8
    std = np.where(std < epsilon, epsilon, std)
    return (X - mean) / std

# Training results

In [6]:
def get_highest_accuracy(y_true, y_pred):
    _, _, thresholds = roc_curve(y_true, y_pred)
    # compute highest accuracy
    thresholds = np.array(thresholds)
    if len(thresholds) > 1000:
        thresholds = np.percentile(thresholds, np.linspace(0, 100, 1001))
    accuracy_scores = []
    for threshold in thresholds:
        accuracy_scores.append(accuracy_score(y_true, y_pred > threshold))

    accuracies = np.array(accuracy_scores)
    return accuracies.max()

In [7]:
# Training parameters
with open('./CNN/params.json', 'r') as f:
    params = json.load(f)
BATCH_SIZE = params['BATCH_SIZE']

In [8]:
def evaluate_EventCNN_on_diphoton_and_ZZ4l(config_path):

    # Read config file
    with open(config_path, 'r') as f:
        config = json.load(f)
    seed = config['seed']
    luminosity = config['luminosity']
    cut_type = config['cut_type']
    model_name = config['model_name']
    GGF_cutflow_file = './Sample/selection_results_GGF_quark_jet.npy'
    VBF_cutflow_file = './Sample/selection_results_VBF_quark_jet.npy'
    
    save_model_name = f'./CNN/CNN_models/last_model_GGF_VBF_CWoLa_{model_name}/'
    try:
        loaded_model = tf.keras.models.load_model(save_model_name)
    except:
        print(f"Model {save_model_name} not found. Skipping this configuration.")
        
    
    n_SR_VBF, n_SR_GGF, n_BR_VBF, n_BR_GGF = compute_nevent_in_SR_BR(GGF_cutflow_file, VBF_cutflow_file, luminosity, cut_type)
    n_events = (int(n_SR_VBF), int(n_SR_GGF), int(n_BR_VBF), int(n_BR_GGF))
    npy_paths = ['./Sample/data/quark_jet_2_cut/pre-processing/remove_photon_case_1/40x40/']
    X_test, y_test = create_test_sample_from(npy_paths, n_events, (0.8, 0.2), seed=seed)
    X_test = pt_normalization(X_test)
    
    y_pred = loaded_model.predict(X_test, batch_size=BATCH_SIZE)
    ACC_aa = get_highest_accuracy(y_test, y_pred)
    AUC_aa = roc_auc_score(y_test, y_pred)

    # ZZ->4l case
    GGF_cutflow_file = './Sample/selection_results_GGF_ZZ_4l_quark_jet.npy'
    VBF_cutflow_file = './Sample/selection_results_VBF_ZZ_4l_quark_jet.npy'
    n_SR_VBF, n_SR_GGF, n_BR_VBF, n_BR_GGF = compute_nevent_in_SR_BR(GGF_cutflow_file, VBF_cutflow_file, luminosity, cut_type)
    n_events = (int(n_SR_VBF), int(n_SR_GGF), int(n_BR_VBF), int(n_BR_GGF))
    npy_paths = ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/']
    X_test, y_test = create_test_sample_from(npy_paths, n_events, (0.8, 0.2), seed=seed)
    X_test = pt_normalization(X_test)
    # Use the same loaded_model as above (same model_name)
    y_pred = loaded_model.predict(X_test, batch_size=BATCH_SIZE)
    ACC_ZZ_4l = get_highest_accuracy(y_test, y_pred)
    AUC_ZZ_4l = roc_auc_score(y_test, y_pred)

    # Write results
    file_name = 'GGF_VBF_CWoLa_summary.csv'
    data_dict = {
                'Di-photon ACC': [ACC_aa],
                'Di-photon AUC': [AUC_aa],
                'ZZ->4l ACC': [ACC_ZZ_4l],
                'ZZ->4l AUC': [AUC_ZZ_4l],
                'Luminosity (fb^-1)': [luminosity],
                'Model type': ['Event-CNN'],
                'Model Name': [model_name],
                }
    
    df = pd.DataFrame(data_dict)
    if os.path.isfile(file_name):
        training_results_df = pd.read_csv(file_name)
        pd.concat([training_results_df, df], ignore_index=True).to_csv(file_name, index=False)
    else:
        df.to_csv(file_name, index=False)

    return ACC_aa, AUC_aa, ACC_ZZ_4l, AUC_ZZ_4l

In [10]:
# for l in [100, 300, 900, 1800, 3000]:
#     for i in range(1, 11):
#         config_path = f'./CNN/config_files/quark_jet_2_cut_L_{l}_eventCNN_pT_norm_remove_photon_case1_config_{i:02}.json'
#         evaluate_EventCNN_on_diphoton_and_ZZ4l(config_path)

# for l in [100, 300, 900, 1800, 3000]:
#     for i in range(1, 11):
#         config_path = f'./CNN/config_files/quark_jet_2_cut_L_{l}_eventCNN_pT_norm_remove_photon_case1_phi_aug_5_config_{i:02}.json'
#         evaluate_EventCNN_on_diphoton_and_ZZ4l(config_path)

for l in [100, 300, 900, 1800, 3000]:
    for i in range(1, 11):
        config_path = f'./CNN/config_files/quark_jet_2_cut_L_{l}_eventCNN_pT_norm_remove_photon_case1_phi_aug_10_config_{i:02}.json'
        evaluate_EventCNN_on_diphoton_and_ZZ4l(config_path)

Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-processing/remove_photon_case_1/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-processing/remove_photon_case_1/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-processing/remove_photon_case_1/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/remove_product_case_5/40x40/']
40/40 [==============================] - 0s 7ms/step
Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-pr

## Particle transformer

In [9]:
MAX_CONSTI = {
    'Jet': 50,
    'Tower': 250,
    'Track': 150,
    'Photon': 2,
    'PHOTON': 2,
    'Lepton': 4,
    'LEPTON': 4,
}

In [10]:
def prepare_feature_from_h5(h5_file, remove_decay_products=False):
    with h5py.File(h5_file, 'r') as f:
        if 'PHOTON' in f.keys():
            decay_product = 'PHOTON'
        elif 'LEPTON' in f.keys():
            decay_product = 'LEPTON'
        else:   
            raise ValueError('No decay product found in the h5 file.')
        event_pt = np.concatenate([f['TOWER/pt'][:], f['TRACK/pt'][:], f[f'{decay_product}/pt'][:]], axis=1)
        event_eta = np.concatenate([f['TOWER/eta'][:], f['TRACK/eta'][:], f[f'{decay_product}/eta'][:]], axis=1)
        event_phi = np.concatenate([f['TOWER/phi'][:], f['TRACK/phi'][:], f[f'{decay_product}/phi'][:]], axis=1)
        event_mask = np.concatenate([f['TOWER/mask'][:], f['TRACK/mask'][:], np.tile([True] * MAX_CONSTI[decay_product], (event_pt.shape[0], 1))], axis=1)

        if remove_decay_products:
            decay_product_eta = f[f'{decay_product}/eta'][:]
            decay_product_phi = f[f'{decay_product}/phi'][:]
            indices = np.where((event_eta[:, :, None] == decay_product_eta[:, None, :]) & (event_phi[:, :, None] == decay_product_phi[:, None, :]))
            event_mask[indices[0], indices[1]] = False

        event_pt[event_mask == False] = float('nan')
        event_eta[event_mask == False] = float('nan')
        event_phi[event_mask == False] = float('nan')

        event_particle_type_0 = np.array([1] * MAX_CONSTI['Tower'] + [0] * MAX_CONSTI['Track'] + [0] * MAX_CONSTI[decay_product])
        event_particle_type_0 = np.tile(event_particle_type_0, (event_pt.shape[0], 1))
        event_particle_type_1 = np.array([0] * MAX_CONSTI['Tower'] + [1] * MAX_CONSTI['Track'] + [0] * MAX_CONSTI[decay_product])
        event_particle_type_1 = np.tile(event_particle_type_1, (event_pt.shape[0], 1))
        event_particle_type_2 = np.array([0] * MAX_CONSTI['Tower'] + [0] * MAX_CONSTI['Track'] + [1] * MAX_CONSTI[decay_product])
        event_particle_type_2 = np.tile(event_particle_type_2, (event_pt.shape[0], 1))

        features = np.stack([event_pt, event_eta, event_phi, event_particle_type_0, event_particle_type_1, event_particle_type_2], axis=-1)

    return features

In [11]:
def create_ParT_test_sample(h5_dirs: list, nevents: tuple, ratios=(0.8, 0.2), seed=0, remove_decay_products=False):
    # h5_dirs: list of npy directories
    # nevents: tuple of (n_VBF_SR, n_VBF_BR, n_GGF_SR, n_GGF_BR)
    # ratios: tuple of (r_train, r_val)


    h5_dir0 = Path(h5_dirs[0])
    data_VBF_SR = prepare_feature_from_h5(h5_dir0 / 'VBF_in_SR.h5', remove_decay_products)
    data_VBF_BR = prepare_feature_from_h5(h5_dir0 / 'VBF_in_BR.h5', remove_decay_products)
    data_GGF_SR = prepare_feature_from_h5(h5_dir0 / 'GGF_in_SR.h5', remove_decay_products)
    data_GGF_BR = prepare_feature_from_h5(h5_dir0 / 'GGF_in_BR.h5', remove_decay_products)
    n_data_VBF_SR = data_VBF_SR.shape[0]
    n_data_VBF_BR = data_VBF_BR.shape[0]
    n_data_GGF_SR = data_GGF_SR.shape[0]
    n_data_GGF_BR = data_GGF_BR.shape[0]
    print(f'Number of events in VBF_SR: {n_data_VBF_SR}, VBF_BR: {n_data_VBF_BR}, GGF_SR: {n_data_GGF_SR}, GGF_BR: {n_data_GGF_BR}')

    n_VBF_SR, n_GGF_SR, n_VBF_BR, n_GGF_BR = nevents
    n_test = 10000
    n_VBF_SR_test = int(n_data_VBF_SR / (n_data_VBF_SR + n_data_VBF_BR) * n_test)
    n_VBF_BR_test = n_test - n_VBF_SR_test
    n_GGF_SR_test = int(n_data_GGF_SR / (n_data_GGF_SR + n_data_GGF_BR) * n_test)
    n_GGF_BR_test = n_test - n_GGF_SR_test


    np.random.seed(seed)
    idx_VBF_SR = np.random.choice(n_data_VBF_SR, n_VBF_SR + n_VBF_SR_test, replace=False)
    idx_VBF_BR = np.random.choice(n_data_VBF_BR, n_VBF_BR + n_VBF_BR_test, replace=False)
    idx_GGF_SR = np.random.choice(n_data_GGF_SR, n_GGF_SR + n_GGF_SR_test, replace=False)
    idx_GGF_BR = np.random.choice(n_data_GGF_BR, n_GGF_BR + n_GGF_BR_test, replace=False)


    idx_VBF_SR_te = idx_VBF_SR[n_VBF_SR:]
    idx_VBF_BR_te = idx_VBF_BR[n_VBF_BR:]
    idx_GGF_SR_te = idx_GGF_SR[n_GGF_SR:]
    idx_GGF_BR_te = idx_GGF_BR[n_GGF_BR:]

    print(f'Preparing dataset from {h5_dirs}')

    new_data_te = np.concatenate([
        data_VBF_SR[idx_VBF_SR_te],
        data_VBF_BR[idx_VBF_BR_te],
        data_GGF_SR[idx_GGF_SR_te],
        data_GGF_BR[idx_GGF_BR_te],
    ], axis=0)
    data_te = new_data_te

    new_label_te = np.zeros(new_data_te.shape[0])
    new_label_te[:n_test] = 1
    label_te = new_label_te

    return  data_te, label_te

def pt_normalization_ParT(X):
    slices = [slice(0, 250), slice(250, 400), slice(400, 402)]
    for i, s in enumerate(slices):
        mean = np.nanmean(X[:, s, 0], axis=(1), keepdims=True)
        std = np.nanstd(X[:, s, 0], axis=(1), keepdims=True)
        mean[np.isnan(mean)] = 0
        std[np.isnan(std)] = 1
        epsilon = 1e-8
        std = np.where(std < epsilon, epsilon, std)

        X[:, s, 0] = (X[:, s, 0] - mean) / std

In [12]:
def evaluate_ParT_on_diphoton_and_ZZ4l(config_path):

    # Read config file
    with open(config_path, 'r') as f:
        config = json.load(f)
    seed = config['seed']
    luminosity = config['luminosity']
    cut_type = config['cut_type']
    model_name = config['model_name']
    training_method = config['training_method']
    remove_decay_products = config['remove_decay_products']
    GGF_cutflow_file = './Sample/selection_results_GGF_quark_jet.npy'
    VBF_cutflow_file = './Sample/selection_results_VBF_quark_jet.npy'
    
    save_model_name = f'./Particle_transformer/ParT_models/last_model_{training_method}_{model_name}/'
    try:
        loaded_model = tf.keras.models.load_model(save_model_name)
    except:
        print(f"Model {save_model_name} not found. Skipping this configuration.")
        
    
    n_SR_VBF, n_SR_GGF, n_BR_VBF, n_BR_GGF = compute_nevent_in_SR_BR(GGF_cutflow_file, VBF_cutflow_file, luminosity, cut_type)
    n_events = (int(n_SR_VBF), int(n_SR_GGF), int(n_BR_VBF), int(n_BR_GGF))
    h5_dirs = ['./Sample/data/quark_jet_2_cut/pre-processing/']
    X_test, y_test = create_ParT_test_sample(h5_dirs, n_events, (0.8, 0.2), seed=seed, remove_decay_products=remove_decay_products)
    pt_normalization_ParT(X_test)
    
    y_pred = loaded_model.predict(X_test, batch_size=BATCH_SIZE)
    ACC_aa = get_highest_accuracy(y_test, y_pred)
    AUC_aa = roc_auc_score(y_test, y_pred)

    # ZZ->4l case
    GGF_cutflow_file = './Sample/selection_results_GGF_ZZ_4l_quark_jet.npy'
    VBF_cutflow_file = './Sample/selection_results_VBF_ZZ_4l_quark_jet.npy'
    n_SR_VBF, n_SR_GGF, n_BR_VBF, n_BR_GGF = compute_nevent_in_SR_BR(GGF_cutflow_file, VBF_cutflow_file, luminosity, cut_type)
    n_events = (int(n_SR_VBF), int(n_SR_GGF), int(n_BR_VBF), int(n_BR_GGF))
    npy_paths = ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/']
    X_test, y_test = create_ParT_test_sample(npy_paths, n_events, (0.8, 0.2), seed=seed, remove_decay_products=remove_decay_products)
    pt_normalization_ParT(X_test)
    # Use the same loaded_model as above (same model_name)
    y_pred = loaded_model.predict(X_test, batch_size=BATCH_SIZE)
    ACC_ZZ_4l = get_highest_accuracy(y_test, y_pred)
    AUC_ZZ_4l = roc_auc_score(y_test, y_pred)

    # Write results
    file_name = 'GGF_VBF_CWoLa_summary.csv'
    data_dict = {
                'Di-photon ACC': [ACC_aa],
                'Di-photon AUC': [AUC_aa],
                'ZZ->4l ACC': [ACC_ZZ_4l],
                'ZZ->4l AUC': [AUC_ZZ_4l],
                'Luminosity (fb^-1)': [luminosity],
                'Model type': ['ParT'],
                'Model Name': [model_name],
                }
    
    df = pd.DataFrame(data_dict)
    if os.path.isfile(file_name):
        training_results_df = pd.read_csv(file_name)
        pd.concat([training_results_df, df], ignore_index=True).to_csv(file_name, index=False)
    else:
        df.to_csv(file_name, index=False)

    return ACC_aa, AUC_aa, ACC_ZZ_4l, AUC_ZZ_4l

In [13]:
for l in [100, 300, 900, 1800, 3000]:
    for i in range(1, 11):
        # config_path = f'./Particle_transformer/config_files/CWoLa_ParT_L{l}_remove_decay_products_pTnorm_config_{i:02}.json'
        # config_path = f'./Particle_transformer/config_files/CWoLa_ParT_L{l}_remove_decay_products_pTnorm_phi_aug_5_config_{i:02}.json'
        config_path = f'./Particle_transformer/config_files/CWoLa_ParT_L{l}_remove_decay_products_pTnorm_phi_aug_10_config_{i:02}.json'
        evaluate_ParT_on_diphoton_and_ZZ4l(config_path)

2025-10-08 10:24:40.180111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 10:24:40.777222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


Number of events in VBF_SR: 286751, VBF_BR: 45436, GGF_SR: 78945, GGF_BR: 83384
Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-processing/']


/tmp/ipykernel_519871/2994601865.py:57: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(X[:, s, 0], axis=(1), keepdims=True)
/home/r10222035/.conda/envs/tf2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2025-10-08 10:25:05.304389: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2025-10-08 10:25:05.305949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


40/40 [==============================] - 7s 134ms/step
Number of events in VBF_SR: 115461, VBF_BR: 17701, GGF_SR: 32970, GGF_BR: 32429
Preparing dataset from ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/']


/tmp/ipykernel_519871/2994601865.py:57: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(X[:, s, 0], axis=(1), keepdims=True)
/home/r10222035/.conda/envs/tf2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


...

40/40 [==============================] - 6s 134ms/step
Number of events in VBF_SR: 286751, VBF_BR: 45436, GGF_SR: 78945, GGF_BR: 83384
Preparing dataset from ['./Sample/data/quark_jet_2_cut/pre-processing/']


/tmp/ipykernel_519871/2994601865.py:57: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(X[:, s, 0], axis=(1), keepdims=True)
/home/r10222035/.conda/envs/tf2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


40/40 [==============================] - 6s 134ms/step
Number of events in VBF_SR: 115461, VBF_BR: 17701, GGF_SR: 32970, GGF_BR: 32429
Preparing dataset from ['./Sample/data/ZZ_4l/quark_jet_2_cut/pre-processing/']


/tmp/ipykernel_519871/2994601865.py:57: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(X[:, s, 0], axis=(1), keepdims=True)
/home/r10222035/.conda/envs/tf2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


40/40 [==============================] - 6s 135ms/step
